In [34]:
import pandas as pd
import numpy as np


In [70]:
df = pd.read_csv("/mnt/hdd/anatkorol/Imagination_in_translation/Data/processed_data/08092025_pilot/participants_log_with_gpt_with_distances_pilot_08092025_.csv")

In [71]:
df

,uid,gt,session,attempt,img_index,request_seed,returned_seed,prompt,negative_prompt,gen,subjective_score,prompt_latency_secs,model_latency_secs,rating_latency_secs,generated_at_utc,ts,clip_cosine_distance,vgg_fc7_distance
0,00aeccd632c742d48a9ffe94da201493,farm_h.jpg,1,1,1,481678699,481678699,Hall,NaN,00aeccd632c742d48a9ffe94da201493_session01_attempt01_img01_seed481678699.png,13.0,10.799,11.655,13.221,2025-09-08T11:16:23.064779+00:00,1757330197,0.561354,0.796981
1,00aeccd632c742d48a9ffe94da201493,farm_h.jpg,1,2,1,481678699,481678699,Public meeting hall,NaN,00aeccd632c742d48a9ffe94da201493_session01_attempt02_img01_seed481678699.png,64.0,11.360,12.852,14.576,2025-09-08T11:17:03.773220+00:00,1757330240,0.453747,0.714026
2,00aeccd632c742d48a9ffe94da201493,farm_h.jpg,1,3,1,481678699,481678699,Public bathroom,NaN,00aeccd632c742d48a9ffe94da201493_session01_attempt03_img01_seed481678699.png,88.0,13.336,12.056,19.006,2025-09-08T11:17:49.615692+00:00,1757330290,0.445245,0.812723
3,00aeccd632c742d48a9ffe94da201493,fountain_l.jpg,2,1,1,1558508275,1558508275,water fountain,NaN,00aeccd632c742d48a9ffe94da201493_session02_attempt01_img01_seed1558508275.png,87.0,8.512,13.288,29.402,2025-09-08T11:18:34.291775+00:00,1757330345,0.207650,0.365267
4,00aeccd632c742d48a9ffe94da201493,fountain_l.jpg,2,2,1,1558508275,1558508275,wishing well,NaN,00aeccd632c742d48a9ffe94da201493_session02_attempt02_img01_seed1558508275.png,60.0,14.189,13.528,20.321,2025-09-08T11:19:36.653968+00:00,1757330402,0.171779,0.508487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,gpt-5,fountain_l.jpg,7,2,1,1558508275,1558508275,"Frontal symmetrical view of an elegant outdoor fountain in a formal garden. The structure is built from pale beige limestone with a semicircular upper basin and a rounded projecting lip. A small central bubbler creates ripples while a smooth silky curtain of water spills over the front into a lower pool, forming white foam at the base. Behind the basin is a raised planter with a row of yellow flowers and a small shrub, backed by a curved stone wall. Tall dark cypress trees flank the scene, and white spherical lamps sit on square pedestals at the ends. A dense green ivy wall fills the background. In the foreground tan rectangular pavers form a neat geometric pattern. Soft even daylight, high detail, realistic texture, serene atmosphere, deep depth of field, no people.",NaN,3096a1302cf348448360769b8420339f_session07_attempt02_img01_seed1558508275.png,NaN,4.922,22.316,19.646,2025-09-12T10:57:50.670793+00:00,1757674690,0.158431,0.334926
187,gpt-5,fountain_l.jpg,7,3,1,1558508275,1558508275,"Symmetrical centered straight-on eye-level photo of a formal semicircular stone fountain in a manicured garden, cream limestone or travertine basin with rounded coping and a small central bubbler, water sheeting evenly over the front lip as a silky curtain into a lower pool with swirling foam, slight long exposure, surrounding terrace of matching light stone with a curved back wall, a narrow bed of yellow flowers, two tall dark cypress trees flanking the center, and white globe lamps on square pedestals, background of dense green ivy, foreground paved with pale herringbone bricks, soft diffuse daylight, ultra realistic, clean classical aesthetic, serene mood, no people.",NaN,3096a1302cf348448360769b8420339f_session07_attempt03_img01_seed1558508275.png,NaN,14.557,21.961,17.320,2025-09-12T10:58:49.370192+00:00,1757674746,0.144122,0.332905
188,gpt-5,dam_l.jpg,8,1,1,1687665244,1687665244,"Photorealistic wide shot in bright midday light of a concrete floodgate structure spanning a shallow marsh channel, front facing, six rectangular culvert openings with metal flap gates, steel sheet pile wing walls, riprap rock bank on the left, muddy shore with dry reeds and driftwood in the right foreground, calm reflective water, sparse shrubs behind the structure, clear pale blue sky, natural earthy colors, no people.",NaN,3096a1302cf348448360769b8420339f

In [36]:
df_gpt['subjective_score'] = np.nan
df_gpt['uid'] = 'gpt-5'
df_gpt.head(3)

,uid,gt,session,attempt,img_index,request_seed,returned_seed,prompt,negative_prompt,gen,subjective_score,prompt_latency_secs,model_latency_secs,rating_latency_secs,generated_at_utc,ts
0,gpt-5,farm_h.jpg,1,1,1,481678699,481678699,"Wide angle daytime view of a cluster of small single story cabins or barracks with white horizontal clapboard siding and red standing seam metal gable roofs, arranged in rows on a gently sloping mown grass field. The nearest long rectangular building is shown in three-quarter view with a cross gable entry porch and open doorway on the left, a row of small high horizontal windows along the side wall, a plain rear side door, red fascia and gutters, and a low concrete foundation. An asphalt road and sidewalk with a curb and simple metal handrail run along the left edge. Identical white and red buildings recede into the midground and background, with a few larger utilitarian structures beyond and a line of dark evergreen trees on the horizon under a bright overcast sky with soft clouds.",NaN,3096a1302cf348448360769b8420339f_session01_attempt01_img01_seed481678699.png,NaN,34.684,18.893,20.119,2025-09-12T10:44:14.667451+00:00,1757673874
1,gpt-5,farm_h.jpg,1,2,1,481678699,481678699,"Daytime eye level wide angle photo of a cluster of small single story white buildings with red standing seam metal gable roofs on a gently rolling grassy hill. The nearest building is centered with white lap siding, a long side wall holding four narrow horizontal rectangular windows, a plain door at the right end, and a shorter perpendicular wing on the left with an open entrance. Several matching structures recede into the distance, an asphalt road and a concrete walkway with metal rail run along the left, the mowed grass is bright green, evergreen trees and low sheds line the far horizon, and the sky is partly cloudy with soft even light, quiet rural campus feel, no people.",NaN,3096a1302cf348448360769b8420339f_session01_attempt02_img01_seed481678699.png,NaN,5.730,17.819,10.490,2025-09-12T10:45:00.719624+00:00,1757673911
2,gpt-5,farm_h.jpg,1,3,1,481678699,481678699,"Wide-angle daytime photo of a simple white single-story building with a red standing-seam metal gable roof and white horizontal siding, seen from left-front three-quarter view on a neatly mowed grassy slope beside a paved road and a metal handrail ramp; four small high rectangular windows line the long wall, a white door sits near the right end, and an open entry bay under a small gable projects on the left; a low concrete foundation, red fascia and roof flashing, and a thin vent pipe on the roof are visible; multiple similar white buildings with matching red roofs recede across the rolling campus into the distance, with scattered evergreen trees along the horizon and a bright sky with soft clouds; natural colors, sharp realistic detail, eye-level perspective, no people.",NaN,3096a1302cf348448360769b8420339f_session01_attempt03_img01_seed481678699.png,NaN,9.915,18.702,10.712,2025-09-12T10:45:42.161298+00:00,1757673952


In [37]:
df = pd.concat([df_ppt, df_gpt], ignore_index=True)

In [40]:
df.to_csv('/mnt/hdd/anatkorol/Imagination_in_translation/Data/processed_data/08092025_pilot/participants_log_with_gpt_08092025_pilot.csv', index=False)

In [16]:
desc_df.to_csv("gpt_descriptions/gpt-5_image_descriptions_only_verbosity-low.csv", index=False)

In [67]:
filename = df_gpt[0:1]['gen']
filename

0    3096a1302cf348448360769b8420339f_session01_attempt01_img01_seed481678699.png
Name: gen, dtype: object

In [68]:
(str(filename)[:37])

'0    3096a1302cf348448360769b8420339f'

In [66]:
int(str(filename)[45:47])

1

In [ ]:
n = goast'

NameError: name 'goast' is not defined

In [56]:
import re
GEN_PATTERN = re.compile(
    r"""^(?P<uid>[0-9a-f]{32})
         _session(?P<session>\d{2})
         _attempt(?P<attempt>\d{2})
         _img(?P<img_index>\d{2})
         (?:_seed\d+)?            # optional _seed123...
         \.(?P<ext>png|jpg|jpeg)$
     """,
    re.IGNORECASE | re.VERBOSE,
)


In [57]:
def parse_gen_name(gen_name: str):
    """Return (uid, session, attempt, img_index, ext, seedless_filename)."""
    name = Path(gen_name).name.strip()
    m = GEN_PATTERN.match(name)
    if not m:
        raise ValueError(f"Unexpected gen filename format: {gen_name}")

    uid      = m["uid"]
    session  = int(m["session"])
    attempt  = int(m["attempt"])
    img_idx  = int(m["img_index"])
    ext      = m["ext"].lower()

    # Build the seedless filename you actually have on disk:
    seedless = f"{uid}_session{session:02d}_attempt{attempt:02d}_img{img_idx:02d}.{ext}"
    return uid, session, attempt, img_idx, ext, seedless

